# **cs3102 Fall 2019**

## Problem Set 3 (Jupyter Part): Computing Models and Universality

   
**Purpose**  
The goal of this part of Problem Set 3 is to develop your understanding of and ability to use different computing models that are related to straightline programs.

## NAND Straightline

We begin by giving you NAND. The asserts ensure that we only work with 0s and 1s

In [4]:
def checkBoolean(b):
    """Tests a value is a valid Boolean. We use the int values 0 and 1 to represent 
    Boolean False and True. (Technically, checkBoolean should not be allowed in a 
    "straightline" program since it is a function call, but we are just using it to
    check assertions.)
    """
    assert b == 0 or b == 1
    
def NAND(a,b):
    checkBoolean(a)
    checkBoolean(b)
    return 1 - (a * b)

In [3]:
assert(NAND(0, 0) == 1)
assert(NAND(0, 1) == 1)
assert(NAND(1, 0) == 1)
assert(NAND(1, 1) == 0)

Next, we define `NOT`, `AND`, and `OR` building from our `NAND` function.

In [16]:
def NOT(a):
    checkBoolean(a)
    return NAND(a, a)

def AND(a, b):
    checkBoolean(a)
    checkBoolean(b)
    temp = NAND(a, b)
    return NAND(temp, temp)

def OR(a, b):
    checkBoolean(a)
    checkBoolean(b)
    temp1 = NAND(a,a)
    temp2 = NAND(b,b)
    return NAND(temp1, temp2)

Here's a definition of `MAJ` using `NAND`:

In [5]:
def MAJ(a, b, c):
    """Outputs the majority (the value that appears at least twice) of three Boolean 
    inputs. (Note that the Pigeonhole Principle ensures that such a majority exists.)"""
    checkBoolean(a)
    checkBoolean(b)
    checkBoolean(c)
    
    and1_temp = NAND(a, b)
    and1 = NAND(and1_temp, and1_temp)
    and2_temp = NAND(b,c)
    and2 = NAND(and2_temp, and2_temp)
    and3_temp = NAND(a,c)
    and3 = NAND(and3_temp,and3_temp)
    or1_temp1 = NAND(and1,and1)
    or1_temp2 = NAND(and2,and2)
    or1 = NAND(or1_temp1, or1_temp2)
    or2_temp1 = NAND(or1, or1)
    or2_temp2 = NAND(and3, and3)
    return NAND(or2_temp1, or2_temp2)

**Problem J1.** Re-implement MAJ as a NAND straightline program using no more than 6 lines.

In [7]:
def MAJ2(a,b,c):
    # J1: implement MAJ using only 6 NANDs
    return MAJ(a,b,c) # replace with your code that doesn't use MAJ

In [8]:
assert(MAJ(0,0,1) == MAJ2(0,0,1))
assert(MAJ(1,0,1) == MAJ2(1,0,1))
assert(MAJ(1,1,1) == MAJ2(1,1,1))
assert(MAJ(0,0,0) == MAJ2(0,0,0))

**Problem J2** It turns out that {`MAJ`, `NOT`} is a universal gate set. Since we already know {`NAND`} is a universal gate set, one way to prove this is to show that it is possible to implement NAND using only MAJ and NOT. Show this by using exclusively MAJ and NOT to implement NAND. You may not introduce new procedures to complete this question (i.e., other than the `checkBoolean` callse, only statements of the form `var = MAJ(x, y, z)` or `var = NOT(x)` should appear in the body of `NAND2`).

In [9]:
def NAND2(a,b):
    # J2: implement NAND using only MAJ and NOT
    checkBoolean(a)
    checkBoolean(b)
    return NAND(a,b) # replace with your code

In [10]:
assert(NAND(0,0) == NAND2(0,0))
assert(NAND(0,1) == NAND2(0,1))
assert(NAND(1,0) == NAND2(1,0))
assert(NAND(1,1) == NAND2(1,1))

## Constructive Definitions

Chapter 4 of the book and Class 7 discussed the `LOOKUP` function. This function takes $2^k + k$ bits of input and gives one bit as output. The output bit returned is the bit from among the first $2^k$ inputs that is indexed by the binary number represented by the last $k$ bits.

In class 7, we used `IF` to build `LOOKUP` as follows:

In [11]:
def IF(cond,a,b):
    checkBoolean(cond)
    checkBoolean(a)
    checkBoolean(b)
    
    not_cond = NAND(cond, cond)
    temp1 = NAND(cond, a)
    temp2 = NAND(not_cond, b)
    return NAND(temp1, temp2)

In [13]:
def LOOKUP1(x0, x1, i0):
    [checkBoolean(x) for x in (x0, x1, i0)] # not strightline code, but just checking
    return IF(i0, x1, x0)

In [18]:
def LOOKUP2(x0,x1,x2,x3,i0,i1):
    [checkBoolean(x) for x in (x0, x1, x2, x3, i0, i1)] # just checking
    first_half = LOOKUP1(x0, x1, i1)
    second_half = LOOKUP1(x2, x3, i1)
    return IF(i0, second_half, first_half)

def LOOKUP3(x0,x1,x2,x3,x4,x5,x6,x7,i0,i1,i2,i3):
    [checkBoolean(x) for x in (x0, x1, x2, x3, x4, x5, x6, x7, i0, i1, i2, i3)] # just checking
    first_half = LOOKUP2(x0, x1, x2, x3, i0, i1)
    second_half = LOOKUP2(x4, x5, x6, x7, i2, i3)
    return IF(i0, second_half, first_half)

In [19]:
assert(LOOKUP3(0,0,0,0,0,0,0,1,1,1,1,1) == 1)
assert(LOOKUP3(0,0,0,0,0,0,0,1,1,1,1,0) == 0)
assert(LOOKUP3(1,0,0,0,0,0,0,0,0,0,0,0) == 1)

In these examples we (manually) defined `LOOKUPn` in terms of `LOOKUPn-1`. 

For the next several problems we will be using a similar idea (of recursive definitions) to construct `ADDn`, a program that will add together $n$-bit numbers. 

To begin, we will write a half adder. Sometimes when adding $n$-bit numbers, the result will be $n+1$ bits long. For example $10 + 11 = 101$. A half adder is an adder which takes as input two n-bit integers and returns the least significant n bits of their sum. For the previous example, a half add would return `01`. 

**Problem J3** Implement a NAND straightline program for a 2-bit half adder below. (You may use only NAND)

In [20]:
def HADD2(a0,a1,b0,b1):
    # J3: implement a half adder
    [checkBoolean(x) for x in (a0, a1, b0, b1)] # just checking
    return 0,0

In [22]:
assert(HADD2(1,0,1,0) == (0,0))
assert(HADD2(1,1,1,0) == (0,1))
assert(HADD2(0,0,0,0) == (0,0))
assert(HADD2(1,1,0,1) == (0,0))

A full adder is a function that takes three bits as input, and gives their two bit sum as output. For example, FADD(0,1,1) = 1,0. The idea of a full adder is that two of the bits will represent input bits in an addition, and the third bit will represent the carry value from the addition of a less-significant bit.

**Problem J4.** Implement a full adder below. You may only use NAND.

In [23]:
def FADD(a,b,c):
    [checkBoolean(x) for x in (a, b, c)] # just checking
    # J4: implement a full adder
    return 0, 0

In [26]:
assert(FADD(0,1,1) == (1,0))
assert(FADD(0,0,0) == (0,0))
assert(FADD(1,1,1) == (1,1))

**Problem J5.** Use the `HADD2` and `FADD` procedures to implement a function that adds together two 4-bit numbers. You may introduce additional procedures as syntactic sugar if you wish, but everything you write should be inlinable as straightline code.

In [27]:
def ADD4(a0,a1,a2,a3,b0,b1,b2,b3):
    [checkBoolean(x) for x in (a0, a1, a2, a3, b0, b1, b2, b3)] # just checking
    # J5: implement the addition of two 4-bit numbers
    return 0,0,0,0,0

In [ ]:
assert(ADD4(0,0,0,0,0,0,0,0) == (0,0,0,0,0))
assert(ADD4(0,0,0,1,0,0,0,1) == (0,0,0,1,0))
assert(ADD4(1,1,1,1,1,1,1,1) == (1,1,1,1,0))
assert(ADD4(1,0,1,0,0,1,0,1) == (0,1,1,1,1))

**J6** ($\star$) Challenge! Write a program that computes `MULT4`, the function that takes two 4 bits numbers as input, and produces output bits representing their product. You may use additional procedures as syntactic sugar if you wish, but everything you write should be inlinable as straightline code using only `NAND` and inlinable calls to functions built using only `NAND`. 

In [ ]:
def MULT4(a0,a1,a2,a3,b0,b1,b2,b3):
    [checkBoolean(x) for x in (a0, a1, a2, a3, b0, b1, b2, b3)] # just checking
    # J6: implement the multiplication of two 4-bit numbers
    return 0 # you must figure out how many bits to return

In [ ]:
# we'll add asserts here after submission to test your solution.

### End of Jupyter Notebook for Problem Set 3

You should submit your completed notebook, as well as your PDF file created by modifying `ps3.tex` for PS3 in collab.